In [2]:
import cv2
import time
import numpy as np
from utils import *

In [38]:
img = cv2.imread("Images/dark-mode.png", 1)

HEIGHT, WIDTH = img.shape[:2]
AREA = HEIGHT * WIDTH

np.save("dimensions", np.array([HEIGHT, WIDTH]))

CELL_AREA_MAX = int(AREA / (9*9)) + 1
CELL_AREA_MIN = CELL_AREA_MAX // 12

show(img)

False

In [39]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 9, 4)
thresh2 = cv2.erode(thresh, KERNEL5)


show(thresh, thresh2)

False

In [40]:
contours, _ = cv2.findContours(thresh2, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

cimg = draw(img, contours)
show(cimg, thresh2)

False

In [55]:
proImgg = img.copy()
proImgr = img.copy()
proImgb = img.copy()

possible_cell_area = []
boards = []

for cnt in contours:

    cur = cv2.contourArea(cnt)
    
    if CELL_AREA_MIN < cur < CELL_AREA_MAX:
        
        draw_on(proImgg, [cnt], thickness=-1)

        simplified = approxPoly(cnt)
        
        if len(simplified) == 4:
            possible_cell_area.append(cur)
            draw_on(proImgr, [simplified], color='r', thickness=-1)
    
    elif CELL_AREA_MIN * 81 < cur < CELL_AREA_MAX * 81:
        
        simplified = approxPoly(cnt)
        
        if len(simplified) == 4:
            boards.append([cur, simplified])


possible_cell_area.sort()

Area, _ = frameArea(possible_cell_area)


outerContour = None

for area, contour in boards:
    if Area * 0.7 < area < Area * 1.3:
        outerContour = contour
        
draw_on(proImgb, [outerContour], thickness=-1)

show(proImgg, proImgr, proImgb)

False

In [12]:
if outerContour is not None:
    
    pt1 = np.float32(sorted(outerContour.reshape(4, 2), key=lambda x: (x[0], x[1])))
    
    pt2 = [[0, 0], [0, OUTER_DIM], [OUTER_DIM, 0], [OUTER_DIM, OUTER_DIM]]
    if pt1[0][1] > pt1[1][1]:
        pt2[0], pt2[1] = pt2[1], pt2[0]
    
    if pt1[2][1] > pt1[3][1]:
        pt2[2], pt2[3] = pt2[3], pt2[2]
    
    pt2 = np.float32(pt2)
    
    np.save("pt1", pt1)
    np.save("pt2", pt2)
    
    M = cv2.getPerspectiveTransform(pt1, pt2)
    new_img = cv2.warpPerspective(thresh, M, (OUTER_DIM, OUTER_DIM))
    
    show(img, new_img)
    
    cv2.imwrite("tmp.png", new_img)
    
else:
    print("No board detected")
